# 剛体の回転運動

吉田勝俊（宇都宮大学）

## 参考情報

- [Pythonで運動方程式を解く(odeint) - Qiita](https://qiita.com/binaryneutronstar/items/ad5efa27fd626826846f)
- [[Python] Numpyの参照、抽出、結合 - Qiita](https://qiita.com/supersaiakujin/items/d63c73bb7b5aac43898a)
- [[Python/matplotlib] FuncAnimationを理解して使う - Qiita](https://qiita.com/osanshouo/items/3c66781f41884694838b)

In [ ]:
import numpy as np                              #数値計算ライブラリ
import numpy.linalg as la                       #線形代数ライブラリ
from math import sin, cos, pi                   #低機能だが計算が速い数学関数
from scipy.integrate import odeint              #常微分方程式ライブラリ
import matplotlib.pyplot as plt                 #描画ライブラリ
from mpl_toolkits.mplot3d import Axes3D         #3次元座標用のモジュール
from mpl_toolkits.mplot3d.art3d import Poly3DCollection #3次元ポリゴン用のモジュール
from matplotlib.animation import FuncAnimation  #アニメーションライブラリ
from matplotlib import rc                       #グラフ調整ライブラリ
#↓Colab用の設定（グラフィックのインライン表示）
%matplotlib inline

### 運動学用の関数

In [ ]:
E3 = np.eye(3)  #3x3単位行列
E4 = np.eye(4)  #4x4単位行列

def CrossMatrix(vec):
    '''
    外積行列
    '''
    return np.array([
        [ 0, -vec[2],  vec[1]],
        [ vec[2],  0, -vec[0]],
        [-vec[1],  vec[0],  0],        
    ])

def RotationMatrix(q):
    '''
    回転行列をオイラーパラメータから作る
    '''
    qvec = q[1:]
    return np.array(
        (2*q[0]**2 - 1)*E3 + 2*np.outer(qvec,qvec) + 2*q[0]*CrossMatrix(qvec)
    )

def q_from_axis_angle(axis, angle):
    '''
    オイラーパラメータを(回転軸，回転角)から作る
    '''
    half = 0.5*angle
    cos_half = cos(half)
    sin_half = sin(half)
    return np.array([
        cos_half,
        axis[0]*sin_half,
        axis[1]*sin_half,
        axis[2]*sin_half,
    ])

def H_Matrix(Rot, vec):
    '''
    同次変換行列を(回転行列，変位ベクトル)から作る
    '''
    H = np.hstack((Rot,np.array([vec]).T))
    H = np.vstack((H,[0,0,0,1]))
    
    return H

### ポリゴンデータの操作

In [ ]:
def polys_H_multiply(polys, H):
  '''
  同次変換行列 H をポリゴンのリスト polys に掛ける
  '''
  POLY_LIST = []
  for poly in polys: #ポリゴン１枚の頂点セット in そのリスト
        
    POLY = []
    for point in poly: #各頂点 in あるポリゴンの頂点セット
      Hpoint = np.hstack((point,[1])) #同次座標化
      Hpoint = np.dot(H,Hpoint)       #同次変換
      point = Hpoint[:-1]             #3次元座標化
      POLY.append(point)
            
    POLY_LIST.append(POLY)
        
  return POLY_LIST

def polys_Trans(polys, r):
  '''
  ポリゴンの平行移動
  '''
  POLY_LIST = []
  for poly in polys: #ポリゴン１枚の頂点セット in そのリスト
        
    POLY = []
    for point in poly: #各頂点 in あるポリゴンの頂点セット
      POLY.append(point + r)

    POLY_LIST.append(POLY)
        
  return POLY_LIST

def polys_plot(ax, polys):
  '''
  ポリゴンの三次元プロット
  '''
  facecolors=['C1','C2','C6','C6','C3']
  for i, p in enumerate(list(polys)):
    pc = Poly3DCollection([p])
    pc.set_alpha(0.7) #alpha first
    pc.set_facecolor(facecolors[i])
    ax.add_collection3d(pc)
  
  ax.set_xlim(-1,1)
  ax.set_ylim(-1,1)
  ax.set_zlim(-1,1)
  ax.set_xlabel('X')
  ax.set_ylabel('Y')
  ax.set_zlabel('Z')

def polys_plot_by_H(ax, polys0, H):
  '''
  ポリゴンの三次元プロット（同次行列から）
  '''
  polys = polys_H_multiply(polys0, H)
  polys_plot(ax, polys)
        
def polys_plot_by_q(ax, polys, q, vec):
  '''
  ポリゴンの三次元プロット（オイラーパラメータ，変位ベクトルから）
  '''
  Rot = RotationMatrix(q)
  H = H_Matrix(Rot,vec)
  polys_plot_by_H(ax, polys, H)

### ポリゴンのデータサンプル

In [ ]:
def polys_triangle(w, d, h):
  '''
  厚みのある二等辺三角形 (幅, 厚, 高)
  '''
  A = np.array([ w/2, -d/2, 0])
  B = np.array([ w/2,  d/2, 0])
  C = np.array([-w/2,  d/2, 0])
  D = np.array([-w/2, -d/2, 0])
  E = np.array([   0, -d/2, h])
  F = np.array([   0,  d/2, h])
    
  verts =[
    [F, B, C],    #前面
    [E, D, A],    #後面
    [A, B, F, E], #斜面
    [C, D, E, F], #斜面
    [A, B, C, D], #底面
  ]
    
  return verts

### お試しプロット

In [ ]:
# 基準姿勢データのサンプル
#【注】描画する形状は厚みのある3角形ですが，その中心は，
#　　　Ｌ字の棒状の物体の重心（真ん中）に取りました．
triangle = polys_triangle(w=1, d=0.1, h=0.5)
triangle = polys_Trans(triangle, [0, 0, -0.5/2])

# 同次行列 H のサンプル
q = q_from_axis_angle( [1,0,0], pi/2 )
r = [0,0,0]
H = H_Matrix( RotationMatrix(q),r )

# 基準姿勢データを同次変換
body0 = triangle
body = polys_H_multiply(body0, H)

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot (1, 1, 1, projection = '3d')

polys_plot_by_H(ax, body0, E4)
polys_plot_by_q(ax, body, q, r)

fig.tight_layout()
plt.show()

## 主軸オイラー方程式

In [ ]:
def EulerEQ(om, diagI):
  '''
  主軸オイラー方程式
  '''
  return -np.array([
    (diagI[2]-diagI[1])*om[1]*om[2]/diagI[0],  
    (diagI[0]-diagI[2])*om[2]*om[0]/diagI[1],  
    (diagI[1]-diagI[0])*om[0]*om[1]/diagI[2],  
  ])

def Kq(q):
  '''
  運動学的方程式（オイラーパラメータ）の係数行列
  '''
  return 0.5*np.array([
    [-q[1], -q[2], -q[3]],
    [ q[0], -q[3],  q[2]],
    [ q[3],  q[0], -q[1]],
    [-q[2],  q[1],  q[0]],
  ])

def State_Equation(x, t, diagI):
  '''
  状態方程式
  '''
  om = x[0:3] #角速度ベクトル
  q  = x[3:]  #オイラーパラメータ

  dom = EulerEQ(om, diagI)  #主軸オイラー方程式
  dq = np.dot(Kq(q), om)    #運動学的方程式

  return np.hstack((dom,dq))

### シミュレーション条件

In [ ]:
body_diagI = np.array([   #主軸慣性モーメント（平べったい物体）
    12, 2, 10
])

body_x0 = np.array([      #状態ベクトルの初期値
    1, 1, -7,             #角速度の初期値
    1, 0, 0, 0,           #オイラーパラメータの初期値
])

body_dt = 0.02            #数値積分の時間ステップ

### 時間応答

In [ ]:
dt = body_dt
x0 = body_x0
diagI = body_diagI

tN = 200 #1200/3
ts = np.linspace(0, 12.9, tN) #時間軸
sol = odeint(State_Equation, x0, ts, args=(diagI,))  #状態方程式を解く

In [ ]:
fig = plt.figure(figsize=(5,2.5*7))
axs = fig.subplots(nrows=7)

for i, ax in enumerate(axs):
    ax.plot(ts, sol[:,i])
    ax.set_xlabel('t')
    if i<3:
        ax.set_ylabel('$\omega_%d$'%(i+1))
    else:
        ax.set_ylabel('$q_%d$'%(i-3))

### ちなみに，角運動量の基準成分や，角速度ベクトルは？

In [ ]:
Ls = []
om_norms = []

for x in sol:
  om = x[0:3] #角速度ベクトル
  q  = x[3:]  #オイラーパラメータ

  R = RotationMatrix(q)
  inertia_tensor = np.diag(diagI)
  L = R.dot(inertia_tensor).dot(om)   #角運動量の基準成分
  Ls.append(L)

  om_norms.append(la.norm(om))

Ls = np.array(Ls)
om_norms = np.array(om_norms)

fig = plt.figure(figsize=(5,2.5*4))
axs = fig.subplots(nrows=4)

for i in range(3):
    ax = axs[i]
    values = Ls[:,i]
    ax.plot(ts, values)
    ax.set_xlabel('t')
    ax.set_ylabel('$L_%d$'%(i+1))

    maxval = max(0, np.max(values))
    minval = min(0, np.min(values))
    ax.set_ylim(1.2*minval,1.2*maxval)

values = om_norms
axs[3].plot(ts, values)
axs[3].set_xlabel('t')
axs[3].set_ylabel(r'$||\omega||$')

maxval = max(0, np.max(values))
minval = min(0, np.min(values))
axs[3].set_ylim(1.2*minval,1.2*maxval)

fig.tight_layout()

- 角運動量ベクトルの基準成分は，計算機誤差の範囲で，定数になっています．
- すなわち，基準フレームから見た角運動量は，理論通り保存しています．

- その一方で，角速度ベクトルの長さは，時間とともに伸縮しています．

#### よくある？誤解
- 「角運動量が保存するから，角速度ベクトルも保存する」という説明は，一般的には，間違いです．
- そうなるのは，空間に固定された軸まわりの回転運動だけです．

## アニメーション

* アニメーションの生成に若干時間を要します．

In [ ]:
fig = plt.figure(figsize=plt.figaspect(1))
ax = fig.add_subplot(1, 1, 1, projection = '3d')

def each_frame(idx):
  ax.cla()
  q  = sol[idx,3:]  #オイラーパラメータ
  polys_plot_by_q(ax, body0, q, [0,0,0])
  fig.tight_layout()

anim = FuncAnimation(
    fig, each_frame, 
    interval=80, frames=len(sol)
)

rc('animation', html='jshtml')
anim